In [147]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [148]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [149]:
# Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.

tesla = yf.Ticker('TSLA')

In [150]:
# Using the ticker object and the function history extract stock information and save it in a dataframe named tesla_data. Set the period parameter to "max" so we get information for the maximum amount of time.

tesla_data = tesla.history(period='max')

In [ ]:
# Reset the index using the reset_index(inplace=True) function on the tesla_data DataFrame and display the first five rows of the tesla_data dataframe using the head function. Take a screenshot of the results and code from the beginning of Question 1 to the results below.

tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
# Use the requests library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm Save the text of the response as a variable named html_data.

html_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm')
html_data = html_data.content
html_data

In [ ]:
# Parse the html data using beautiful_soup using parser i.e html5lib or html.parser. Make sure to use the html_data with the content parameter as follow html_data.content.

soup = BeautifulSoup(html_data, 'html.parser')
print(soup.prettify())

In [154]:
# Using BeautifulSoup or the read_html function extract the table with Tesla Revenue and store it into a dataframe named tesla_revenue. The dataframe should have columns Date and Revenue.

tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
data = soup.find_all('table', {'class': 'historical_data_table table'})[1].find('tbody').find_all('tr')

for row in data:
    col = row.find_all('td')
    date = col[0].text
    revenue = col[1].text

    tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({'Date' : [date], 'Revenue': [revenue]})])

In [ ]:
# Execute the following line to remove the comma and dollar sign from the Revenue column.

tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(',|\\$',"", regex=True)

In [156]:
# Execute the following lines to remove an null or empty strings in the Revenue column.
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.index = range(0, len(tesla_revenue['Revenue']))
tesla_revenue.tail()

Use Webscraping to Extract GME Revenue Data

In [164]:
# Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is GME.

gme_data = yf.Ticker('GME')
gme_data = gme_data.history(period='max')
gme_data.reset_index(inplace=True)

In [ ]:
# Use the requests library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. Save the text of the response as a variable named html_data_2.

html_data_2 = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html')
html_data_2 = html_data_2.content
print(html_data_2)

In [ ]:
# Parse the html data using beautiful_soup using parser i.e html5lib or html.parser.

soup = BeautifulSoup(html_data_2, 'html.parser')
print(soup.prettify())

In [ ]:
# Using BeautifulSoup or the read_html function extract the table with GameStop Revenue and store it into a dataframe named gme_revenue. The dataframe should have columns Date and Revenue. Make sure the comma and dollar sign is removed from the Revenue column.

gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
data = soup.find_all('table')[1].find('tbody').find_all('tr')

for row in data:
    col = row.find_all('td')
    date = col[0].text
    revenue = col[1].text

    gme_revenue = pd.concat([gme_revenue, pd.DataFrame({'Date': [date], 'Revenue': [revenue]})])

gme_revenue.index = range(0, len(gme_revenue['Date']))


gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',|\\$',"", regex=True)

gme_revenue.head()

In [ ]:
# Display the last five rows of the gme_revenue dataframe using the tail function. Take a screenshot of the results.

gme_revenue.tail()

Plot the graph

In [ ]:
# Use the make_graph function to graph the Tesla Stock Data, also provide a title for the graph. Note the graph will only show data upto June 2021.

make_graph(revenue_data=tesla_revenue, stock_data=tesla_data, stock='Tesla')

In [ ]:
# Use the make_graph function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the make_graph function is make_graph(gme_data, gme_revenue, 'GameStop'). Note the graph will only show data upto June 2021.

make_graph(revenue_data=gme_revenue, stock_data=gme_data, stock='GameStop')